# A Pypher Overview

This notebook is an introduction to Pypher, what it can do, and how it can help manage your Cypher from within Python.

## Install the Pypher package from pypi

In [16]:
!pip install python_cypher

## Import what you need

For these examples we will need the `Pypher` object, the Pypher factory object `__`, and the `Param` object

In [41]:
from pypher import Pypher, __
from pypher.builder import Param


def cprint(pypher):
    c = str(pypher)
    p = pypher.bound_params
    print('Cypher:')
    print(str(pypher))

    if p:
        print('\nBound Params:')
        print(dict(p))
    

## Sample Pypher Queries

Lets write a few simple Cypher queries, but using Pypher.

In [42]:
p = Pypher()
p.Match.node('a').relationship('r').node('b').RETURN('a', 'b', 'r')

cprint(p)

Cypher:
MATCH (a)-[r]-(b) RETURN a, b, r


In [43]:
p = Pypher()
p.MATCH.node('u', labels='User').RETURN.user

cprint(p)

Cypher:
MATCH (u:`User`) RETURN user


In this example a complex Cypher query is built. The Pypher automatically binds the parameters passed into the functions.

In [44]:
p.reset()
p.OPTIONAL.MATCH.node('user', 'User').rel('FRIENDS_WITH').node('friend', 'User')
# continue later
p.WHERE.user.__id__ == 1234
p.RETURN(__.user, __.count('friend').alias('number_of_friends'))

cprint(p)

Cypher:
OPTIONAL MATCH (user:`User`)-[FRIENDS_WITH]-(friend:`User`) WHERE user.`id` = $NEO_e6a72_0 RETURN user, count($NEO_e6a72_1) AS $NEO_e6a72_2

Bound Params:
{'$NEO_e6a72_0': 1234, '$NEO_e6a72_1': 'friend', '$NEO_e6a72_2': 'number_of_friends'}


We can also manually bind parameters using the `Param` object or by calling the `.bind_param` method on the `Pypher` instances.

In [46]:
p = Pypher()

name = Param('my_name', 'Mark')
p.CREATE.node(name=name).RETURN.node()

cprint(p)

Cypher:
CREATE ( {`name`: $my_name}) RETURN ()

Bound Params:
{'$my_name': 'Mark'}


As you can see in both the generated Cypher and the bound_params, `my_name` is used because it was defined.

In [47]:
p = Pypher()
name = p.bind_param('Mark', 'my_param')
p.CREATE.node(name=name).RETURN.node()

cprint(p)

Cypher:
CREATE ( {`name`: $my_param}) RETURN ()

Bound Params:
{'$my_param': 'Mark'}


These next few examples were taken from [https://github.com/Readify/Neo4jClient/wiki/cypher-examples](https://github.com/Readify/Neo4jClient/wiki/cypher-examples)

### Create a user, only if they don't already exist

In [48]:
p = Pypher()

p.MERGE.node('user', labels='User', Id=456).ON_CREATE.user.SET(__.user.__Name__ == 'Jim')

cprint(p)

Cypher:
MERGE (user:`User` {`Id`: $Id42b01_0}) ON_CREATE user SET user.`Name` = $NEO_42b01_1

Bound Params:
{'$Id42b01_0': 456, '$NEO_42b01_1': 'Jim'}


### Create a user and relate them to an existing one

In [49]:
p = Pypher()

p.MATCH.node('invitee', labels='User').WHERE.invitee.__id__ == 123
p.CREATE.node('invitee').rel_out(labels='INVITED').node('invited', lables='User')

cprint(p)

Cypher:
MATCH (invitee:`User`) WHERE invitee.`id` = $NEO_58bd0_0 CREATE (invitee)-[:`INVITED`]->(invited {`lables`: $lables58bd0_1})

Bound Params:
{'$NEO_58bd0_0': 123, '$lables58bd0_1': 'User'}


### Relate two existing users

This example does a few notable things:

* It breaks up the query building across a few lines. This is useful for when you want do things like conditionally build a query
* It runs Pypher Statement objects as functions. When done this way multiple arguments can be passed in.
* It uses the Pypher factory to create anonymous chains that are passed into the statement functions.
* It accesses properties via the double undersore syntax. `user.__id__` is equivilent to `user.property('id')`

In [50]:
p = Pypher()

p.MATCH(__.node('user1', labels='User'), __.node('user2', labels='User')) # line is getting long
p.WHERE(__.user1.__Id__ == 123, __.user2.__Id__ == 456)
p.CREATE.node('user1').relationship(direction='out', labels='FRIENDS_WITH').node('user2')

cprint(p)

Cypher:
MATCH (user1:`User`), (user2:`User`) WHERE user1.`Id` = $NEO_716ef_0, user2.`Id` = $NEO_716ef_1 CREATE (user1)-[:`FRIENDS_WITH`]->(user2)

Bound Params:
{'$NEO_716ef_0': 123, '$NEO_716ef_1': 456}


### Update a single property on a user

This example shows how to access a property via the `property` method.

In [54]:
p = Pypher()

p.MATCH.node('user', 'User').WHERE.user.property('id') == 123
p.SET(__.user.property('Age') == 25)

cprint(p)

Cypher:
MATCH (user:`User`) WHERE user.`id` = $NEO_f4755_0 SET user.`Age` = $NEO_f4755_1

Bound Params:
{'$NEO_f4755_0': 123, '$NEO_f4755_1': 25}


## Take a Look at Partial Objects

In Pypher, Partial objects allow for complex query to be handled in another object with its own interface. Pypher comes with one called `Case` that handles the Cypher swith case syntax.

In [57]:
from pypher.partial import Case

c = Case('')
c.WHEN(__.n.__eyes__ == 'blue', 1).WHEN(__.n.__age__ < 40, 2).ELSE(3)

p = Pypher()
p.MATCH.node().RETURN(c).alias('Result')

cprint(p)

Cypher:
MATCH () RETURN CASE  WHEN n.`eyes` = $NEO_37f9d_0 THEN 1 WHEN n.`age` < $NEO_37f9d_1 THEN 2 ELSE 3 END AS $NEO_37f9d_2

Bound Params:
{'$NEO_37f9d_0': 'blue', '$NEO_37f9d_1': 40, '$NEO_37f9d_2': 'Result'}
